In [ ]:
import pandas as pd

TRAIN_FILE = "/content/drive/MyDrive/Projects/Ongoing Projects/Lung Outcome Prediciton_P20250048/Data/Binarize_With_Median/OS/TH-2Y/Test_LungCT_Diagnosis_And_NSCLC_Radiogenomics/train.xlsx"
TEST_FILE = "/content/drive/MyDrive/Projects/Ongoing Projects/Lung Outcome Prediciton_P20250048/Data/Binarize_With_Median/OS/TH-2Y/Test_LungCT_Diagnosis_And_NSCLC_Radiogenomics/test.xlsx"
# =========================
# Load data
# =========================
train = pd.read_excel(TRAIN_FILE)
test = pd.read_excel(TEST_FILE)

In [ ]:
# Pseudo-labeling with GridSearch, Final Models = LogisticRegression + DecisionTree
import numpy as np

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2, f_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_selection import (
    SelectKBest, f_classif, chi2, mutual_info_classif,
    RFE, SelectFromModel, VarianceThreshold, SelectPercentile,
    GenericUnivariateSelect, SelectFpr, SelectFdr, SelectFwe
)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier
)
import warnings

import shap
import os
warnings.filterwarnings("ignore")

id_col = "PatientID"
outcome_col = "Outcome"
feature_cols = [c for c in train.columns if c not in [id_col, outcome_col]]

# Split train
train_labeled = train[train[outcome_col].notnull()].copy()
train_unlabeled = train[train[outcome_col].isnull()].copy()

X_labeled = train_labeled[feature_cols]
y_labeled = train_labeled[outcome_col].astype(int)
X_unlabeled = train_unlabeled[feature_cols]

X_test = test[feature_cols]
y_test = test[outcome_col].astype(int)

# =========================
# Preprocessing
# =========================
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, feature_cols)]
)
k=10
# =========================
# Feature selection
# =========================
feature_selectors = {

        'ANOVA_F': SelectKBest(score_func=f_classif, k=k),
        # 'Mutual_Info': SelectKBest(score_func=mutual_info_classif, k=k),
        'RFE_LogReg': RFE(LogisticRegression(random_state=42, max_iter=1000), n_features_to_select=k),
        'Random_Forest_Importance': SelectFromModel(RandomForestClassifier(random_state=42, n_estimators=100), max_features=k),
        # 'Extra_Trees_Importance': SelectFromModel(ExtraTreesClassifier(random_state=42, n_estimators=100), max_features=k),
        # 'L1_Regularization': SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', random_state=42), max_features=k),
        'Variance_Threshold': VarianceThreshold(threshold=0.1)
}


# =========================
# Pseudo-label classifiers + grids
# =========================
classifiers = {
    "logreg": (LogisticRegression(max_iter=500),
            {"clf__C": [0.01, 0.1, 1, 10]}),
    "rf": (RandomForestClassifier(),
           {"clf__n_estimators": [50, 100], "clf__max_depth": [None, 5, 10]}),
    # "svc": (SVC(probability=True),
    #         {"clf__C": [0.1, 1], "clf__kernel": ["linear", "rbf"]}),
    # "nb": (GaussianNB(), {}),
    # "knn": (KNeighborsClassifier(),
    #         {"clf__n_neighbors": [3, 5, 7]}),
    # "dt": (DecisionTreeClassifier(),
    #        {"clf__max_depth": [None, 5, 10]}),
    # "gb": (GradientBoostingClassifier(),
    #        {"clf__n_estimators": [50, 100]}),
    "ab": (AdaBoostClassifier(),
           {"clf__n_estimators": [50, 100]}),
    # "xgb": (XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
    #         {"clf__n_estimators": [50, 100]}),
    # "lgbm": (LGBMClassifier(verbose=-1),
    #          {"clf__n_estimators": [50, 100]})
}
# =========================
# Final interpretable models
# =========================
final_models = {

    "logreg": (LogisticRegression(max_iter=500),
            {"clf__C": [0.01, 0.1, 1, 10]}),
    "rf": (RandomForestClassifier(),
           {"clf__n_estimators": [50, 100], "clf__max_depth": [None, 5, 10]}),
    "svc": (SVC(probability=True),
            {"clf__C": [0.1, 1], "clf__kernel": ["linear", "rbf"]}),
    "nb": (GaussianNB(), {}),
    "knn": (KNeighborsClassifier(),
            {"clf__n_neighbors": [3, 5, 7]}),
    "dt": (DecisionTreeClassifier(),
           {"clf__max_depth": [None, 5, 10]}),
    "gb": (GradientBoostingClassifier(),
           {"clf__n_estimators": [50, 100]}),
    "ab": (AdaBoostClassifier(),
           {"clf__n_estimators": [50, 100]}),
    "xgb": (XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
            {"clf__n_estimators": [50, 100]}),
    # "lgbm": (LGBMClassifier(),
    #          {"clf__n_estimators": [50, 100]})
}

# =========================
# Metric functions
# =========================
def compute_metrics(model, X, y):
    preds = model.predict(X)
    proba = model.predict_proba(X)[:,1]
    return {
        "accuracy": accuracy_score(y, preds),
        "f1": f1_score(y, preds),
        "roc_auc": roc_auc_score(y, proba),
        "precision": precision_score(y, preds),
        "recall": recall_score(y, preds)
    }

def crossval_metrics(model, X, y, cv=5):
    metrics = {}
    scorers = {
        "accuracy": "accuracy",
        "f1": "f1",
        "roc_auc": "roc_auc",
        "precision": "precision",
        "recall": "recall"
    }
    for m, sc in scorers.items():
        scores = cross_val_score(model, X, y, cv=cv, scoring=sc)
        metrics[f"val_{m}_mean"] = np.mean(scores)
        metrics[f"val_{m}_std"] = np.std(scores)
    return metrics


results = []

save_dir = "/content/drive/MyDrive/Projects/Ongoing Projects/Lung Outcome Prediciton_P20250048/Arman/Results/Feature Importance/"
os.makedirs(save_dir, exist_ok=True)

for sel_name, selector in feature_selectors.items():
    for clf_name, (clf, clf_grid) in classifiers.items():
        print(f"Pseudo-labeling with FS={sel_name}, CLF={clf_name}")

        pseudo_pipe = Pipeline([
            ('pre', preprocessor),
            ('fs', selector),
            ('clf', clf)
        ])

        if clf_grid:
            grid_pseudo = GridSearchCV(pseudo_pipe, clf_grid, cv=3, scoring="roc_auc", n_jobs=-1)
            grid_pseudo.fit(X_labeled, y_labeled)
            best_pseudo = grid_pseudo.best_estimator_
        else:
            pseudo_pipe.fit(X_labeled, y_labeled)
            best_pseudo = pseudo_pipe

        preds_unlabeled = best_pseudo.predict(X_unlabeled)

        # Extended training data
        train_pseudo = pd.concat([
            train_labeled,
            train_unlabeled.assign(Outcome=preds_unlabeled)
        ])
        X_train_full = train_pseudo[feature_cols]
        y_train_full = train_pseudo[outcome_col].astype(int)

        # Final models
        for final_name, (final_clf, final_grid) in final_models.items():
            print("Final Model is : "+final_name)
            final_pipe = Pipeline([
                ('pre', preprocessor),
                ('fs', SelectKBest(f_classif, k=10)),  # fixed FS for final model
                ('clf', final_clf)
            ])
            grid_final = GridSearchCV(final_pipe, final_grid, cv=5, scoring='roc_auc', n_jobs=-1)
            grid_final.fit(X_train_full, y_train_full)
            best_final = grid_final.best_estimator_

            # Test metrics
            test_metrics = compute_metrics(best_final, X_test, y_test)
            # Validation metrics
            val_metrics = crossval_metrics(best_final, X_train_full, y_train_full, cv=5)

            res = {
                "FS": sel_name,
                "PseudoCLF": clf_name,
                "FinalModel": final_name,
                "BestFinalParams": str(grid_final.best_params_)
            }
            res.update({f"test_{k}": v for k, v in test_metrics.items()})
            res.update(val_metrics)
            results.append(res)

            # =========================
            # SHAP feature importance
            # =========================

            # extract fitted model and selected feature names
            fs_step = best_final.named_steps['fs']
            selected_idx = fs_step.get_support(indices=True)
            selected_features = [feature_cols[i] for i in selected_idx]

            model_only = best_final.named_steps['clf']

            X_sample = X_train_full.sample(20, random_state=42)  # to speed up
            X_proc = best_final.named_steps['pre'].transform(X_sample)
            X_proc_sel = fs_step.transform(X_proc)

            # Assume best_final is your trained AdaBoostClassifier

            explainer = shap.KernelExplainer(model_only.predict_proba, X_proc_sel)
            shap_values = explainer.shap_values(X_proc_sel)

            # selected_features = feature_cols
            # model_only = best_final.named_steps['clf']
            # explainer = shap.Explainer(model_only, X_train_full)
            # shap_values = explainer(X_train_full)

            # SHAP values for binary classification
            # shap_vals = shap_values.values

            # Mean absolute SHAP toward each class
            feature_names = X_sample.columns
            class0_importance = np.abs(shap_values[0]).mean(axis=1)
            class1_importance = np.abs(shap_values[1]).mean(axis=1)
            print(selected_features)
            print(class0_importance)
            print(class1_importance)

            df_imp = pd.DataFrame({
                "Feature": selected_features,
                "Mean SHAP Class0": class0_importance,
                "Mean SHAP Class1": class1_importance
            }).sort_values("Mean SHAP Class1", ascending=False)

            save_path = os.path.join(save_dir, f"shap_importance_final={final_name}_fs={sel_name}_pl={clf_name}.csv")
            df_imp.to_csv(save_path, index=False)

            # =========================
            # Save results table
            # =========================
            results_df = pd.DataFrame(results)
            results_df.to_csv(
                os.path.join(save_dir, "all_results_y2.csv"), index=False
            )